In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Installations

In [ ]:
!pip install tf_slim

In [ ]:
!pip install pycocotools
!pip install lvis
!pip install numba 

## Download and extract TensorFlow Model Garden or cd to it if it exists

In [ ]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

## Compile Protobufs and Install the Object Detection Package

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
# %%bash 
# cd models/research
# pip install .

In [ ]:
%%bash 
cd models/research
cp object_detection/packages/tf2/setup.py .
python -m pip install .

## Set Environment Variabes

In [ ]:
os.environ['PYTHONPATH'] += ":/kaggle/working/models"

import sys
sys.path.append("/kaggle/working/models")

## Import Packages

In [ ]:
import numpy as np
import os
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import json
import pandas as pd
import glob
import os.path as osp
from path import Path
import datetime
import random
import shutil
from io import StringIO, BytesIO
from PIL import Image
from IPython.display import display
import re

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
sns.set(rc={"font.size":9,"axes.titlesize":15,"axes.labelsize":9,
            "axes.titlepad":11, "axes.labelpad":9, "legend.fontsize":7,
            "legend.title_fontsize":7, 'axes.grid' : False})

from sklearn.model_selection import train_test_split

## Import object detection module
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils
from object_detection.protos.string_int_label_map_pb2 import StringIntLabelMap, StringIntLabelMapItem
from object_detection.utils import config_util
from object_detection.builders import model_builder

from google.protobuf import text_format

import tarfile
from numba import cuda 

## Patches

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

## Load and Infer Using a Pre-trained Model


## Loading Label Map

In [ ]:
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Download Model Files

In [ ]:
pretrained_dir = "/kaggle/working/training_job/model/"

if not os.path.exists(pretrained_dir):
    os.makedirs(pretrained_dir)
    print('Pretrainined Model Directory:', pretrained_dir)

## Choosing Pre-trained Model & Downloading Weight Files
- https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In [ ]:
## EfficientDet Configurations

MODELS_CONFIG = {
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d4': {
        'model_name': 'efficientdet_d4_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d4_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d4_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d5': {
        'model_name': 'efficientdet_d5_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d5_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d5_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d6': {
        'model_name': 'efficientdet_d6_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d6_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d6_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d7': {
        'model_name': 'efficientdet_d7_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d7_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d7_coco17_tpu-32.tar.gz',
    }
}

## Choosing D1 here for this tutorial
chosen_model = 'efficientdet-d1'
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [ ]:
model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'+pretrained_checkpoint
# pretrained_dir = "/kaggle/working/models/research/object_detection/pretrained"

!wget {model_url}
!tar -xf {pretrained_checkpoint}
!mv {model_name}/ {pretrained_dir}
!rm {pretrained_checkpoint}

model_dir = os.path.join(pretrained_dir, model_name, "saved_model")
print("Pre-trained model directory", model_dir)
model = tf.saved_model.load(str(model_dir))

## Display the Output Structure

In [ ]:
model.signatures['serving_default'].output_dtypes

In [ ]:
model.signatures['serving_default'].output_shapes

## Helper Functions

In [ ]:
def run_inference_for_single_image(model, image):
    '''
    Add a wrapper function to call the model, and cleanup the outputs:
    '''
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    # Handle models with masks:
    if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                  output_dict['detection_masks'], output_dict['detection_boxes'],
                   image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                           tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    return output_dict

def show_inference(model, image_path):
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(Image.open(image_path))
    # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)
    # Visualization of the results of a detection.
    vis_utils.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

    return image_np
    
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: a file path.

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np, boxes, classes, scores, category_index,
                    figsize=(12, 16), image_name=None):
    """Wrapper function to visualize detections.

    Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
    """
    image_np_with_annotations = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
    if image_name:
        plt.imsave(image_name, image_np_with_annotations)
    else:
        plt.imshow(image_np_with_annotations)
        
        
def plot_img(img, size=(18, 18), is_rgb=True, title="", cmap='gray'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)

## Run the Model on Sample Images

In [ ]:
!wget https://cdn.pixabay.com/photo/2015/03/26/09/43/city-690158_960_720.jpg
!wget https://cdn.pixabay.com/photo/2017/08/05/23/31/people-2586656_960_720.jpg
!wget https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg
    
!wget https://cdn.pixabay.com/photo/2020/02/07/15/33/girl-4827500_960_720.jpg
TEST_IMAGE_PATHS = [
                    'girl-4827500_960_720.jpg',
                    'Naxos_Taverna.jpg',
                    'city-690158_960_720.jpg'
                   ]

In [ ]:
for image_path in TEST_IMAGE_PATHS:
    image = show_inference(model, image_path)
    os.remove(image_path)
    plot_img(image)
    plt.savefig('{}_viz.png'.format(image_path))
    plt.show()

## Delete Model and Release GPU Memory

In [ ]:
del model
device = cuda.get_current_device()
device.reset()

## TFRecord Creation

## Create Ouput TFRecord Folder

In [ ]:
tfr_output_dir = "/kaggle/working/training_job/tfrecords/"

if not os.path.exists(tfr_output_dir):
    os.makedirs(tfr_output_dir)
    print('TFRecord Directory:', tfr_output_dir)

In [ ]:
%%bash
python models/research/object_detection/dataset_tools/create_coco_tf_record.py --logtostderr \
--train_image_dir="../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled" \
--test_image_dir="../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled" \
--val_image_dir="../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled" \
--train_annotations_file="../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/train_annotations.json" \
--testdev_annotations_file="../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_annotations.json" \
--val_annotations_file="../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_annotations.json" \
--output_dir="/kaggle/working/training_job/tfrecords/"

## Creating the Label Map File

In [ ]:
def convert_classes(classes, start=1):
    msg = StringIntLabelMap()
    for id, name in enumerate(classes, start=start):
        msg.item.append(StringIntLabelMapItem(id=id, name=name))
    text = str(text_format.MessageToBytes(msg, as_utf8=True), 'utf-8')
    return text

labels =  [
            "Aortic_enlargement",
            "Atelectasis",
            "Calcification",
            "Cardiomegaly",
            "Consolidation",
            "ILD",
            "Infiltration",
            "Lung_Opacity",
            "Nodule_Mass",
            "Other_lesion",
            "Pleural_effusion",
            "Pleural_thickening",
            "Pneumothorax",
            "Pulmonary_fibrosis"
            ]

txt = convert_classes(labels)
print(txt)
with open('/kaggle/working/training_job/label_map.pbtxt', 'w') as f:
    f.write(txt)

## Preparing Train Job Folder

In [ ]:
model_files_dir = "/kaggle/working/training_job/model_files/"

if not os.path.exists(model_files_dir):
    os.makedirs(model_files_dir)
    print('Model Files Directory:', model_files_dir)

In [ ]:
## Download pretrained weights
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar} -P {model_files_dir}
tar = tarfile.open(os.path.join(model_files_dir, pretrained_checkpoint))
tar.extractall()
tar.close()

In [ ]:
## Download base training configuration file

download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/'+base_pipeline_file
!wget {download_config} -P {model_files_dir}

In [ ]:
## Get the Total Number of Classes

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())


val_record_fname = '/kaggle/working/training_job/tfrecords/coco_val.record*'
train_record_fname = '/kaggle/working/training_job/tfrecords/coco_train.record*'
label_map_pbtxt_fname = '/kaggle/working/training_job/label_map.pbtxt'

pipeline_file = os.path.join(model_files_dir, base_pipeline_file)
fine_tune_checkpoint = os.path.join(pretrained_dir, model_name, 'checkpoint/ckpt-0')
num_classes = get_num_classes(label_map_pbtxt_fname)
print("No. of Classes", num_classes)
print("Checkpoint File Path", fine_tune_checkpoint)

## Configuring the Model Training Pipeline File

## File Structure

In [ ]:
# SSD with EfficientNet-b1 + BiFPN feature extractor,
# shared box predictor and focal loss (a.k.a EfficientDet-d1).
# See EfficientDet, Tan et al, https://arxiv.org/abs/1911.09070
# See Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from an EfficientNet-b1 checkpoint.
#
# Train on TPU-8

# model {
#   ssd {
#     inplace_batchnorm_update: true
#     freeze_batchnorm: false
#     num_classes: 90
#     add_background_class: false
#     box_coder {
#       faster_rcnn_box_coder {
#         y_scale: 10.0
#         x_scale: 10.0
#         height_scale: 5.0
#         width_scale: 5.0
#       }
#     }
#     matcher {
#       argmax_matcher {
#         matched_threshold: 0.5
#         unmatched_threshold: 0.5
#         ignore_thresholds: false
#         negatives_lower_than_unmatched: true
#         force_match_for_each_row: true
#         use_matmul_gather: true
#       }
#     }
#     similarity_calculator {
#       iou_similarity {
#       }
#     }
#     encode_background_as_zeros: true
#     anchor_generator {
#       multiscale_anchor_generator {
#         min_level: 3
#         max_level: 7
#         anchor_scale: 4.0
#         aspect_ratios: [1.0, 2.0, 0.5]
#         scales_per_octave: 3
#       }
#     }
#     image_resizer {
#       keep_aspect_ratio_resizer {
#         min_dimension: 640
#         max_dimension: 640
#         pad_to_max_dimension: true
#         }
#     }
#     box_predictor {
#       weight_shared_convolutional_box_predictor {
#         depth: 88
#         class_prediction_bias_init: -4.6
#         conv_hyperparams {
#           force_use_bias: true
#           activation: SWISH
#           regularizer {
#             l2_regularizer {
#               weight: 0.00004
#             }
#           }
#           initializer {
#             random_normal_initializer {
#               stddev: 0.01
#               mean: 0.0
#             }
#           }
#           batch_norm {
#             scale: true
#             decay: 0.99
#             epsilon: 0.001
#           }
#         }
#         num_layers_before_predictor: 3
#         kernel_size: 3
#         use_depthwise: true
#       }
#     }
#     feature_extractor {
#       type: 'ssd_efficientnet-b1_bifpn_keras'
#       bifpn {
#         min_level: 3
#         max_level: 7
#         num_iterations: 4
#         num_filters: 88
#       }
#       conv_hyperparams {
#         force_use_bias: true
#         activation: SWISH
#         regularizer {
#           l2_regularizer {
#             weight: 0.00004
#           }
#         }
#         initializer {
#           truncated_normal_initializer {
#             stddev: 0.03
#             mean: 0.0
#           }
#         }
#         batch_norm {
#           scale: true,
#           decay: 0.99,
#           epsilon: 0.001,
#         }
#       }
#     }
#     loss {
#       classification_loss {
#         weighted_sigmoid_focal {
#           alpha: 0.25
#           gamma: 1.5
#         }
#       }
#       localization_loss {
#         weighted_smooth_l1 {
#         }
#       }
#       classification_weight: 1.0
#       localization_weight: 1.0
#     }
#     normalize_loss_by_num_matches: true
#     normalize_loc_loss_by_codesize: true
#     post_processing {
#       batch_non_max_suppression {
#         score_threshold: 1e-8
#         iou_threshold: 0.5
#         max_detections_per_class: 100
#         max_total_detections: 100
#       }
#       score_converter: SIGMOID
#     }
#   }
# }

# train_config: {
#   fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED/ckpt-0"
#   fine_tune_checkpoint_version: V2
#   fine_tune_checkpoint_type: "classification"
#   batch_size: 128
#   sync_replicas: true
#   startup_delay_steps: 0
#   replicas_to_aggregate: 8
#   use_bfloat16: true
#   num_steps: 300000
#   data_augmentation_options {
#     random_horizontal_flip {
#     }
#   }
#   data_augmentation_options {
#     random_scale_crop_and_pad_to_square {
#       output_size: 640
#       scale_min: 0.1
#       scale_max: 2.0
#     }
#   }
#   optimizer {
#     momentum_optimizer: {
#       learning_rate: {
#         cosine_decay_learning_rate {
#           learning_rate_base: 8e-2
#           total_steps: 300000
#           warmup_learning_rate: .001
#           warmup_steps: 2500
#         }
#       }
#       momentum_optimizer_value: 0.9
#     }
#     use_moving_average: false
#   }
#   max_number_of_boxes: 100
#   unpad_groundtruth_tensors: false
# }

# train_input_reader: {
#   label_map_path: "PATH_TO_BE_CONFIGURED/label_map.txt"
#   tf_record_input_reader {
#     input_path: "PATH_TO_BE_CONFIGURED/train2017-?????-of-00256.tfrecord"
#   }
# }

# eval_config: {
#   metrics_set: "coco_detection_metrics"
#   use_moving_averages: false
#   batch_size: 1;
# }

# eval_input_reader: {
#   label_map_path: "PATH_TO_BE_CONFIGURED/label_map.txt"
#   shuffle: false
#   num_epochs: 1
#   tf_record_input_reader {
#     input_path: "PATH_TO_BE_CONFIGURED/val2017-?????-of-00032.tfrecord"
#   }
# }

## Define Parameters

In [ ]:
## Train Images = 3296
## 3296/8 = 412
## Val Images = 1098
## 1098/8 ~= 138

## Training Configurations
num_epochs = 5
num_steps = 412*num_epochs
num_eval_steps = 138
batch_size = 8
print("Number of Steps:", num_steps)

## Modify the Pipeline Configuration File

In [ ]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

print('Updading the Training Configuration file')
with open(pipeline_file) as f:
    s = f.read()
    
with open(pipeline_file, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")',
               'input_path: "{}"'.format(train_record_fname), s)
    
    s = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")',
               'input_path: "{}"'.format(val_record_fname), s)

    # label_map_path
    s = re.sub('label_map_path: ".*?"',
               'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    
    #fine-tune checkpoint type
    s = re.sub('fine_tune_checkpoint_type: "classification"',
               'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

In [ ]:
!cat {pipeline_file}

## Train Custom TF2 Object Detector

In [ ]:
model_dir = "/kaggle/working/training_job/model_files/training/"

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    print('Training Files:', model_dir)

In [ ]:
## Checking GPU Load
!nvidia-smi

## Start Training

In [ ]:
!python /kaggle/working/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

## Model Evaluation

## Perform Evaluation using Model Main

In [ ]:
import glob
import os
os.environ['PYTHONPATH'] += ":/kaggle/working/models"
import sys
sys.path.append("/kaggle/working/models")

In [ ]:
model_dir = "/kaggle/working/training_job/model_files/training/"
pipeline_file = glob.glob(os.path.join('/kaggle/working/training_job/model_files/', '*.config'))[0]
pipeline_file

In [ ]:
## Stop this cell manually
!python /kaggle/working/models/research/object_detection/model_main_tf2.py \
     --pipeline_config_path={pipeline_file} \
     --model_dir={model_dir} \
     --checkpoint_dir={model_dir} \
     --run_once=True